# LU-Zerlegung: Vor- / Rückwärtseinsetzen

## Ziel

In dieser Lektion implementieren wir das Lösungsverfahren $A x = b$ unter der Annahme, dass eine $LR$-Zerlegung $A = L R$ der Matrix $A$ bereits gegeben ist.
 
 **Abgabe**: Sie geben als Praktikumsbericht ihren Code in Matlab oder Python (oder C, oder ...) ab. Für die Bewertung ist ausschlaggebend, dass Sie sich ersichtlich mit der Aufgabe auseinandergesetzt haben, unabhängig davon, ob der Code am Ende läuft oder nicht. Wir werden allerdings im nachfolgenden Praktikum darauf aufbauen.

## Theorie

Lineare Gleichungssysteme $Ax = b$ für $A \in \mathbb{R}^{n \times n}, b \in \mathbb{R}^n$ können durch Vorwärts- bzw. Rückwärtseinsetzen gelöst werden, wenn die Matrix $A=R$ eine obere bzw. $A=L$ eine untere Dreiecksmatrix ist. Sie implementieren und testen in diesem Praktikum die beiden Algorithmen. Das *Vorwärtseinsetzen* ist Algorithmus 1 im Skript. Rückwärtseinsetzen funktioniert analog.

Je nach Kontext kann zudem eine effiziente Implementierung entscheidend sein, sowohl memory-efficient (kein unnötiges Belegen von Speicherplatz) wie auch time-efficient. Im einzelnen bedeutet das

 * Die LU-Zerlegung wird in-place ausgeführt, wobei die ursprüngliche Matrix $A$ überschrieben wird. Die resultierende Matrix LR enthält im Dreieck links unten die Matrix $L$, ohne die Diagonalelemente und im Dreieck oben rechts die Matrix R (inklusive Diagonale). Dies enspricht der Vorgehensweise im Unterricht.
 * Es werden keine Zeilenvertauschungen explizit durchgeführt, stattdessen wird mit einem Indexvektor gearbeitet. 
 


## 1.3 Aufgaben

### Aufgabe 1 => Nicht gefordert

1. Beschreiben Sie das Vorwärtseinsetzen als Pseudocode, analog zu Algorithmus $1$ im Skript.
2. Beschreiben Sie in Pseudocode das Vor- und Rückwärtseinsetzen, wenn die LU-Zerlegung in *einer* Matrix $LR$ gegeben ist. Beachten Sie, dass der Vektor $x$ beim Rückwärtseinsetzen überschrieben werden kann. Drücken Sie die Operationen soweit möglich als Skalarprodukte aus.
3. Beschreiben Sie die in-place LU-Zerlegung ohne Zeilentausch als Pseudocode. 

### Aufgabe 2

Implementieren Sie eine Funktion **fbSubs** für das Vorwärts- und Rückwärtseinsetzen. Versuchen Sie, so wenig Schleifen wie möglich zu verwenden!

In [16]:
import numpy as np

"""
forwardBackwardSubs: Vorwärts- und Rückwärtseinsetzen
in: 
 - Matrix LR, die das Ergebnis einer LU-Zerlegung enthält
 - Vektor b: rechte Seite des LGS
out: Lösung x des LGS
"""  






def fbSubs(LR, b):
    # code
    n = len(b)   #extrahiert die Grösse der MAtrix. bestimmt anzahl gleichungen
   
     # Vorwärtseinsetzen
    y = np.zeros(n)
    y[0]=b[0]
    for i in range(1,n):
        y[i] = b[i] - np.dot(LR[i][:i], y[:i])
   
 
    # Rückwärtseinsetzen
    x = np.zeros(n)
    x[n-1]= y[n-1]/LR[n-1][n-1]
    for j in range(n-2, -1, -1):  
        x[j] = (y[j] - np.dot(LR[j][j+1:], x[j+1:]))/LR[j][j]
 
    return x
 

"""
def fbSubs(LR, b):
    n= len(b)
    y = np.zeros(n)
   
    y[0]=b[0]
    for i in range(2,n):
        
        res =0
        for j in range(1,i-1):
            res +=LR[i,j]*y[j]
            
        y[i] = b[i]-res

    x = np.zeros(n)
   
    x[n-1] = y[n-1]/LR[n-1,n-1]
    
    ii = 0
    for i in range(1,n-1):
        ii = n -i
        j = i+1
        res=0
        for j in range(j,n):
            res+=LR[ii,j]*x[j]
        x[i]=(1/LR[i,i])*res
    
    return x



    
    
"""


#test fbsubs
#testmatrix = np.array([[1.,2.,2.,2.],[2.,1.,2.,2.],[2.,2.,1.,2.]])
#btest = np.array([5,5,6,5])
#print(fbSubs(testmatrix,btest))


'\ndef fbSubs(LR, b):\n    n= len(b)\n    y = np.zeros(n)\n   \n    y[0]=b[0]\n    for i in range(2,n):\n        \n        res =0\n        for j in range(1,i-1):\n            res +=LR[i,j]*y[j]\n            \n        y[i] = b[i]-res\n\n    x = np.zeros(n)\n   \n    x[n-1] = y[n-1]/LR[n-1,n-1]\n    \n    ii = 0\n    for i in range(1,n-1):\n        ii = n -i\n        j = i+1\n        res=0\n        for j in range(j,n):\n            res+=LR[ii,j]*x[j]\n        x[i]=(1/LR[i,i])*res\n    \n    return x\n\n\n\n    \n    \n'

### Aufgabe 3

Testen Sie **fbSubs** zuerst mit dem Zahlenbeispiel aus dem Unterricht (wo Sie alles schrittweise nachvollziehen können) und dann mit dem folgenden Testcode, den Sie auch variieren dürfen, bis Sie sicher sind, dass alles wie gewünscht funktioniert. Sie müssen sich im nachfolgenden Praktikum auf **fbSubs** verlassen können!

In [17]:
#Test LR
n = 7 # Dimension der Koeffizientenmatrix
for k in range(10):                      # Testläufe
    LR = np.array( np.random.rand(n,n) )   # zufällige Matrix LR 
    rhs = np.array(np.random.rand(n))    # zufällige rechte Seite des LGS
    x = fbSubs(LR, rhs)   # Aufruf der eigenen Funktion
    
    L,R = np.tril(LR,-1)+np.identity(n), np.triu(LR) # L und R extrahieren
    
    assert( np.linalg.norm(rhs - L@R@x) < 1e-10)     # Test, mit numerischer Toleranz

# Implementierung Gauss-Algorithmus mit Spaltenpivotisierung

## Ziel

 In diesem Praktikum implementieren wir das Lösungsverfahren $A x = b$ inklusive $LU$-Zerlegung $P A = L R$ der Matrix $A$ mit Spaltenpivotisierung.
 
 **Abgabe**: Sie geben als Praktikumsbericht ihren Code in Matlab oder Python (oder C, oder ...) ab. Für die Bewertung ist ausschlaggebend, dass Sie sich ersichtlich mit der Aufgabe auseinandergesetzt haben, unabhängig davon, ob der Code am Ende läuft oder nicht. Sie können den Code direkt in das notebook schreiben oder in einer Entwicklungsumgebung Ihrer Wahl. 

## Theorie

Ein lineares Gleichungssystem $Ax = b$ kann gelöst werden, indem die Matrix $A$ in eine untere bzw. obere Dreiecksmatrix zerlegt wird: $A = L R$, falls keine Zeilenvertauschungen notwendig sind bzw. $P A = L R$ mit der zusätzlichen *Permutationsmatrix* $P$, falls Zeilen vertauscht werden. 

Diesen Prozess nennt man auf Deutsch $LR$-Zerlegung (englisch: $LU$-factorization, für *lower, upper*). Der Algorithmus ist eine kleine Erweiterung des Gauss-Algorithmus (Algorithmus 2 im Skript). $R$ ist die gewohnte Zeilenstufenform, in $l_{ij}$ merken wir uns zusätzlich, mit welchem Faktor Zeile $i$ von Zeile $j$ subtrahiert wurde. Die Beschreibung des Vorgehens entnehmen Sie dem Unterricht.

Sind Zeilenvertauschungen notwendig (oder auf Grund einer Pivotstrategie erwünscht), so führt die Permutationsmatrix $P$ Buch über diese Vertauschungen: alle Zeilenvertauschungen in $A$ werden auch in der Matrix $P$ übernommen.

Je nach Kontext kann zudem eine effiziente Implementierung entscheidend sein, sowohl memory-efficient (kein unnötiges Belegen von Speicherplatz) wie auch time-efficient. Im einzelnen bedeutet das

 * Die LU-Zerlegung wird *in-place* ausgeführt, wobei die ursprüngliche Matrix $A$ überschrieben wird. Die resultierende Matrix LR enthält im Dreieck links unten die Matrix $L$, ohne die Diagonalelemente und im Dreieck oben rechts die Matrix R (inklusive Diagonale). Dies enspricht der Vorgehensweise im Unterricht.
 * Zeilenvertauschungen können formal durch eine Permutationsmatrix $P$ erfasst werden. Es gilt dann $L R = P A$ und man hat wegen $A x = b$ <=> $P A x = P b$ die beiden LGS 
 
     $L \cdot y = P \cdot b$    und    $R \cdot x = y$

zu lösen. Um die Matrixmultiplikation mit der Permutationsmatrix zu sparen  arbeitet man stattdessen mit einem *Indexvektor* $i$, der zu Beginn auf $i = [0, 1, ..., n-1]$ initialisiert wird. Werden Zeilen in $A$ getauscht, so tauscht man auch die entsprechenden Einträge in $i$ und löst am Ende die beiden LGS

$L \cdot y = b[i]$     und    $R \cdot x = y$

Ziel dieses Praktikums ist eine Funktion **linsolve**, die LGS $A \cdot x = b$ mit Hilfe einer *in-place* *LU-Zerlegung* mit Spaltenpivotisierung löst. Wir verwenden die Ergebnisse des letzten Prakikums:

## 2.3 Aufgaben

### Aufgabe 1

Implementieren Sie die *LU-Zerlegung*: die Funktion **LU** nimmt als Input eine quadratische Matrix $A$ und gibt Dreiecksmatrizen $L, R$ zurück, so dass $L R = A$ gilt. Im ersten Schritt lassen wir die Pivotisierung weg. Der Indexvektor **idx** bleibt dann unverändert.

In [3]:
# LU-Zerlegung der quadratischen Matrix A    
# in: quadratische Matrix A
#out: 
# - A wird überschrieben, unteres Dreieck = L (ohne Diagonale), oberes Dreieck = R
# - idx: Indexvektor der Zeilenvertauschungen
def LU(A):
    m = A.shape[0]
    idx = np.array(range(m))    
    #print("M", m, "idx: ", idx, "A: ", A)

    for k in range(0,m):


        for i in range(k+1,m):
            if(A[k][k] == 0):
                print("Pivot is zero")
            A[i][k]= A[i][k]/A[k][k]
 
            for j in range(k+1,m):
                A[i][j] = A[i][j] - A[i][k]*A[k][j]

    return A, idx

Testen Sie **LU** mit zufällig erzeugten Matrizen.  

In [10]:
n = 7
#test LU
for k in range(1000):
    A = np.array( np.random.rand(n,n) )   # zufällige Matrix A erzeugen
    LR, idx = LU(A.copy())                  # LU-Zerlegung von A   
    L,R = np.tril(LR,-1)+np.identity(n), np.triu(LR)  # Matrizen L, R extrahieren  
    assert( np.linalg.norm(L@R - A[idx]) < 1e-8)

### Aufgabe 2

Erstellen Sie eine Funktion **linsolve(A, b)**, die für eine Matrix $A$ und einen Vektor $b$ das LGS $A x = b$ mit Hilfe der *LR-Zerlegung* und dem Vorwärts- und Rückwärtseinsetzen löst.

In [18]:
# lineares Gleichungssystem A*x = b lösen.
def linsolve(A, b):
    M_LU, idx = LU(A)
    res = fbSubs(M_LU, b)
    return res

Testen Sie **linsolve** mit zufällig erzeugten Matrizen

In [20]:
import numpy.linalg as lin
#test linsolve
for k in range(1000):
    A = np.random.rand(n,n)
    rhs = np.random.rand(n)
    x = linsolve(A.copy(), rhs)   
    assert( lin.norm(rhs - A @ x) < 1e-10)

[-1.7286001   4.13354084  5.24000698 -4.6033939  -2.59744641 -0.2472625
 -2.54286394]
[-1.24870134  0.71084418  0.55947951  1.05807194 -0.29659349  0.75425546
 -1.92363622]
[ 1.06216919 -0.20602433 -0.30913219  0.92149334  0.08184018 -0.68640131
  0.41637002]
[ 2.46584867 -2.63642835 -1.94906579  1.68057894 -1.419689    1.57649174
  1.25152964]
[-7.01556796 -7.69806062  4.76901114  4.55455066 -4.62286373  1.99907917
  6.88228877]
[ 1.0691009   0.27136111 -1.19825082  0.17864247 -0.58630742  0.3517227
  0.68370387]
[-2.48496819 -2.76017976  4.18414852  3.30324711  0.90405033 -2.53838991
  2.09067203]
[ 0.33080138  1.08036004 -0.43420521 -0.1482697   0.24402963 -0.36955739
  0.08193186]
[ 1.06760814 -0.36554241 -0.62954259  0.09088072  0.77704024 -0.58406281
  0.30404958]
[-1.21572468 -0.27275432  0.44929447 -0.34550182 -0.2383376   2.16608449
  1.19255422]
[  8.18883224 -14.99490214  11.40328667  -1.6679475    0.36228798
  -9.08786477   3.44450387]
[ 0.79833365  0.60514842  0.52946833  

Mit der folgenden Funktion *rndCond(n, cond)* können Sie zufällige $n \times n$ Matrizen mit vorgegebener Konditionszahl *cond* erzeugen. 

In [21]:
import numpy.linalg as lin
import numpy.random as rnd

# random orthogonal matrix
def rndOrtho(n):
    S = rnd.rand(n,n)
    S = S - S.T
    O = lin.solve(S - np.identity(n), S + np.identity(n))
    return O

# random matrix with specified condition number
def rndCond(n, cond):    
    d = np.logspace(-np.log10(cond)/2, np.log10(cond)/2,n);
    A = np.diag(d)
    U,V = rndOrtho(n), rndOrtho(n)
    return U@A@V.T

Verwenden Sie diese, um Ihre Implementation von *linsolve* zu testen. Berechnen Sie den relativen Fehler $\frac{|\Delta_x|}{|x|}$ und vergleichen Sie mit dem Ergebnis von *linsolve* aus der Bibliothek *numpy.linalg*. 

In [ ]:
for k in range(N):
    A = rndCond(n, 1e14)
    # code

### Aufgabe 3

Implementieren Sie die Spaltenpivotisierung und wiederholen Sie die Testläufe aus Aufgabe 2